# TP1: Transfer Learning for Medical Image Classification

**Day 3 - AI for Sciences Summer School**

**Instructor:** Raphael Cousin

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/racousin/ai_for_sciences/blob/main/day3/tp1.ipynb)

---


## The BloodMNIST Challenge

**BloodMNIST** is a medical image dataset from the MedMNIST collection:
- **17,092 images** of blood cells (28x28 pixels, RGB)
- **8 classes** of blood cell types
- Real microscopy images from blood smear analysis

```
                    BLOOD SMEAR ANALYSIS
    ╔═══════════════════════════════════════════════╗
    ║                                               ║
    ║     🔴  Red Blood Cells (various types)       ║
    ║     ⚪  White Blood Cells (various types)     ║
    ║     🟡  Platelets                             ║
    ║                                               ║
    ╚═══════════════════════════════════════════════╝
              ↓ Microscopy Image ↓
            ┌─────────────────────┐
            │     28x28 RGB       │
            │       Image         │
            └─────────────────────┘
                      ↓
              AI Classification
                      ↓
           "Neutrophil" (class 2)
```

---

## What You'll Learn

1. How to **explore and visualize** medical image datasets
2. How **data augmentation** improves model robustness
3. How to build a **simple CNN baseline** from scratch
4. How to use **transfer learning** with a frozen backbone
5. How to **fine-tune** a complete pre-trained model
6. When to use each approach based on your data and constraints

---

# Part 1: Setup and Data Exploration

## Setup

In [ ]:
# Install required packages
!pip install -q medmnist torchvision timm

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
import medmnist
from medmnist import INFO
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("Setup complete!")

## Load the BloodMNIST Dataset

In [ ]:
# Dataset information
data_flag = 'bloodmnist'
info = INFO[data_flag]
n_classes = len(info['label'])

print("BloodMNIST Dataset Information")
print("=" * 50)
print(f"Task: {info['task']}")
print(f"Number of channels: {info['n_channels']}")
print(f"Number of classes: {n_classes}")
print(f"Image size: {info['n_channels']} x 28 x 28")
print(f"\nClass labels:")
for i, label in info['label'].items():
    print(f"  {i}: {label}")

In [ ]:
# Load datasets with basic transforms
DataClass = getattr(medmnist, info['python_class'])

# Basic transform: just convert to tensor and normalize
transform_basic = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load train, validation, and test sets
train_dataset = DataClass(split='train', transform=transform_basic, download=True)
val_dataset = DataClass(split='val', transform=transform_basic, download=True)
test_dataset = DataClass(split='test', transform=transform_basic, download=True)

print(f"\nDataset sizes:")
print(f"  Training:   {len(train_dataset):,} images")
print(f"  Validation: {len(val_dataset):,} images")
print(f"  Test:       {len(test_dataset):,} images")
print(f"  Total:      {len(train_dataset) + len(val_dataset) + len(test_dataset):,} images")

## Visualize Sample Images

Before building any model, it's essential to **look at your data**. This helps you understand:
- What the images look like
- How different classes appear
- Potential challenges (image quality, class similarity, etc.)

In [ ]:
# Get class names
class_names = list(info['label'].values())

# Function to denormalize images for display
def denormalize(tensor):
    """Convert normalized tensor back to displayable image."""
    return tensor * 0.5 + 0.5

# Show random samples from each class
fig, axes = plt.subplots(2, 4, figsize=(12, 6))
fig.suptitle('Sample Images from Each Blood Cell Class', fontsize=14, fontweight='bold')

# Get samples for each class
for class_idx in range(8):
    # Find images of this class
    class_indices = [i for i, (_, label) in enumerate(train_dataset) if label[0] == class_idx]
    
    if class_indices:
        # Get a random sample
        sample_idx = class_indices[np.random.randint(len(class_indices))]
        image, label = train_dataset[sample_idx]
        
        # Plot
        ax = axes[class_idx // 4, class_idx % 4]
        img_display = denormalize(image).permute(1, 2, 0).numpy()
        ax.imshow(img_display)
        ax.set_title(f'{class_idx}: {class_names[class_idx]}', fontsize=10)
        ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Analyze class distribution
train_labels = [label[0] for _, label in train_dataset]
unique, counts = np.unique(train_labels, return_counts=True)

fig, ax = plt.subplots(figsize=(10, 5))
colors = plt.cm.Set3(np.linspace(0, 1, 8))
bars = ax.bar(class_names, counts, color=colors, edgecolor='gray')

ax.set_xlabel('Blood Cell Type', fontsize=11)
ax.set_ylabel('Number of Images', fontsize=11)
ax.set_title('Class Distribution in Training Set', fontsize=12, fontweight='bold')
ax.tick_params(axis='x', rotation=45)
ax.grid(True, alpha=0.3, axis='y')

# Add count labels on bars
for bar, count in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20,
            f'{count}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print(f"\nClass imbalance ratio: {max(counts)/min(counts):.1f}x")
print(f"Most common: {class_names[np.argmax(counts)]} ({max(counts)} samples)")
print(f"Least common: {class_names[np.argmin(counts)]} ({min(counts)} samples)")

### Question 1: Understanding the Data

Look at the sample images and class distribution:

1. **Visual inspection**: Which cell types look most similar to each other? Which look most distinct?

2. **Class imbalance**: Is the dataset balanced? How might this affect model training?

3. **Image quality**: The images are only 28x28 pixels. What challenges might this create for classification?

4. **For your research**: What medical imaging data do you work with? What are its characteristics?

---

# Part 2: Data Augmentation

**Data augmentation** artificially increases training data by applying random transformations:
- Helps models generalize better
- Reduces overfitting
- Simulates real-world variations

For medical images, we must choose augmentations that make **biological sense**:
- Rotation: Blood cells can appear at any angle
- Flipping: Cells are symmetric
- Color jitter: Accounts for staining variations
- NOT recommended: Extreme distortions that change cell morphology

In [ ]:
# Let's visualize different augmentation techniques

# Get a sample image
sample_image, sample_label = train_dataset[0]

# Define different augmentations
augmentations = {
    'Original': transforms.Compose([]),
    'Horizontal Flip': transforms.RandomHorizontalFlip(p=1.0),
    'Vertical Flip': transforms.RandomVerticalFlip(p=1.0),
    'Rotation (30)': transforms.RandomRotation(30),
    'Color Jitter': transforms.ColorJitter(brightness=0.3, contrast=0.3),
    'Random Crop': transforms.RandomResizedCrop(28, scale=(0.8, 1.0)),
}

# Visualize
fig, axes = plt.subplots(2, 3, figsize=(12, 8))
fig.suptitle('Data Augmentation Examples', fontsize=14, fontweight='bold')

for idx, (name, aug) in enumerate(augmentations.items()):
    ax = axes[idx // 3, idx % 3]
    
    if name == 'Original':
        img_aug = sample_image
    else:
        # Apply augmentation
        img_aug = aug(denormalize(sample_image))
        if not isinstance(img_aug, torch.Tensor):
            img_aug = transforms.ToTensor()(img_aug)
    
    # Display
    if name == 'Original':
        img_display = denormalize(img_aug).permute(1, 2, 0).numpy()
    else:
        img_display = img_aug.permute(1, 2, 0).numpy()
        img_display = np.clip(img_display, 0, 1)
    
    ax.imshow(img_display)
    ax.set_title(name, fontsize=11)
    ax.axis('off')

plt.tight_layout()
plt.show()

### Exercise 1: Design Your Augmentation Pipeline

Create an augmentation pipeline appropriate for blood cell images. Consider:
- Which transformations make biological sense?
- What intensity of augmentation is appropriate?
- Should you combine multiple augmentations?

In [ ]:
# TODO: Design your augmentation pipeline!

transform_train = transforms.Compose([
    # Add your augmentations here!  # <-- Modify this!
    # Hint: Consider RandomHorizontalFlip, RandomRotation, ColorJitter
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Test transform (no augmentation)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

print("Your augmentation pipeline:")
print(transform_train)

### Question 2: Augmentation Choices

1. Why might **random rotation** be a good augmentation for blood cells?

2. Why might **extreme stretching or shearing** be a bad augmentation for cell classification?

3. How does **color jitter** help with medical images that may have staining variations?

4. What augmentations would be appropriate (or inappropriate) for your research domain?

In [ ]:
# Reload datasets with your augmentation
train_dataset_aug = DataClass(split='train', transform=transform_train, download=True)
val_dataset_test = DataClass(split='val', transform=transform_test, download=True)
test_dataset_test = DataClass(split='test', transform=transform_test, download=True)

# Create data loaders
BATCH_SIZE = 64

train_loader = DataLoader(train_dataset_aug, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"Data loaders created:")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches:   {len(val_loader)}")
print(f"  Test batches:  {len(test_loader)}")

---

# Part 3: Baseline - Simple CNN from Scratch

Before using pre-trained models, let's establish a **baseline** with a simple CNN.

This helps us understand:
- What accuracy can we achieve with a small, custom model?
- How much improvement do pre-trained models provide?

```
Simple CNN Architecture:

Input (3x28x28) → Conv(32) → ReLU → MaxPool
              → Conv(64) → ReLU → MaxPool  
              → Conv(128) → ReLU → AdaptivePool
              → Flatten → Linear(8) → Output
```

In [ ]:
class SimpleCNN(nn.Module):
    """A simple CNN for blood cell classification."""
    
    def __init__(self, num_classes=8):
        super().__init__()
        
        self.features = nn.Sequential(
            # First conv block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Second conv block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Third conv block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)  # Global average pooling
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Create model
model_scratch = SimpleCNN(num_classes=8).to(device)

# Count parameters
total_params = sum(p.numel() for p in model_scratch.parameters())
trainable_params = sum(p.numel() for p in model_scratch.parameters() if p.requires_grad)

print(f"Simple CNN created!")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")

## Training Utilities

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in tqdm(loader, desc='Training', leave=False):
        inputs = inputs.to(device)
        labels = labels.squeeze().long().to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / total, 100. * correct / total


def evaluate(model, loader, criterion, device):
    """Evaluate the model."""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.squeeze().long().to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / total, 100. * correct / total


def train_model(model, train_loader, val_loader, epochs, lr, model_name):
    """Full training loop with history tracking."""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_acc = 0
    
    print(f"\nTraining {model_name}")
    print("=" * 60)
    
    for epoch in range(epochs):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        scheduler.step()
        
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"Epoch {epoch+1:3d}/{epochs}: "
                  f"Train Loss={train_loss:.4f}, Train Acc={train_acc:.1f}%, "
                  f"Val Loss={val_loss:.4f}, Val Acc={val_acc:.1f}%")
        
        if val_acc > best_acc:
            best_acc = val_acc
    
    print(f"\nBest Validation Accuracy: {best_acc:.2f}%")
    return history

## Train the Baseline CNN

In [ ]:
# Train the simple CNN
EPOCHS_BASELINE = 20
LR_BASELINE = 0.001

history_scratch = train_model(
    model_scratch, train_loader, val_loader,
    epochs=EPOCHS_BASELINE, lr=LR_BASELINE, model_name="Simple CNN (from scratch)"
)

In [ ]:
# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(history_scratch['train_acc'], label='Train', linewidth=2)
ax1.plot(history_scratch['val_acc'], label='Validation', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy (%)')
ax1.set_title('Simple CNN - Accuracy', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(history_scratch['train_loss'], label='Train', linewidth=2)
ax2.plot(history_scratch['val_loss'], label='Validation', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title('Simple CNN - Loss', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Question 3: Analyzing the Baseline

Look at the training curves:

1. Is the model **overfitting**? (Gap between train and validation accuracy)

2. Has the model **converged**? (Validation accuracy plateaued)

3. What's the **final validation accuracy**? Is this good enough for clinical use?

4. What could we do to **improve** this baseline? (More layers? Regularization? More data?)

---

# Part 4: Transfer Learning - Frozen Backbone

**Transfer learning** uses models pre-trained on large datasets (like ImageNet with 14M images).

**Strategy 1: Frozen Backbone**
- Use pre-trained feature extractor (frozen = no gradient updates)
- Only train a new classification head
- Fast training, less data needed

```
┌────────────────────────────────┐    ┌─────────────────┐
│   Pre-trained ResNet18         │    │  New Classifier │
│   (FROZEN - no training)       │ →  │   (TRAINED)     │
│   Features from ImageNet       │    │   8 classes     │
└────────────────────────────────┘    └─────────────────┘
```

In [ ]:
# For pre-trained models, we need 224x224 images with ImageNet normalization
transform_pretrained_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_pretrained_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Reload datasets with new transforms
train_dataset_pt = DataClass(split='train', transform=transform_pretrained_train, download=True)
val_dataset_pt = DataClass(split='val', transform=transform_pretrained_test, download=True)
test_dataset_pt = DataClass(split='test', transform=transform_pretrained_test, download=True)

train_loader_pt = DataLoader(train_dataset_pt, batch_size=32, shuffle=True, num_workers=2)
val_loader_pt = DataLoader(val_dataset_pt, batch_size=32, shuffle=False, num_workers=2)
test_loader_pt = DataLoader(test_dataset_pt, batch_size=32, shuffle=False, num_workers=2)

print("Data loaders for pre-trained models created!")
print(f"Image size: 224x224 (upscaled from 28x28)")

### Exercise 2: Create a Frozen Backbone Model

Load a pre-trained ResNet18 and:
1. **Freeze** all backbone parameters (set `requires_grad = False`)
2. **Replace** the final classification layer with a new one for 8 classes

In [ ]:
# Load pre-trained ResNet18
model_frozen = models.resnet18(weights='IMAGENET1K_V1')

# TODO: Freeze all backbone parameters
# Hint: Loop through model.parameters() and set requires_grad = False
# <-- Add your code here!



# TODO: Replace the final fully connected layer
# The original fc layer is: model.fc with in_features = 512, out_features = 1000 (ImageNet classes)
# We need out_features = 8 (our blood cell classes)
num_features = model_frozen.fc.in_features
model_frozen.fc = nn.Linear(num_features, 8)  # This layer will be trainable by default

model_frozen = model_frozen.to(device)

# Check parameters
total_params = sum(p.numel() for p in model_frozen.parameters())
trainable_params = sum(p.numel() for p in model_frozen.parameters() if p.requires_grad)

print(f"ResNet18 (Frozen Backbone)")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Frozen parameters: {total_params - trainable_params:,}")
print(f"  Trainable ratio: {100*trainable_params/total_params:.2f}%")

### Question 4: Understanding Frozen Backbones

1. **Why freeze the backbone?** What advantage does this provide?

2. How many parameters are trainable vs frozen? What does this mean for training speed?

3. The backbone was trained on **ImageNet** (dogs, cars, etc.). Why might it still work for **blood cells**?

4. When might a frozen backbone **not work well**?

In [ ]:
# Train the frozen backbone model
EPOCHS_FROZEN = 10
LR_FROZEN = 0.001  # Can use higher LR since we're only training the classifier

history_frozen = train_model(
    model_frozen, train_loader_pt, val_loader_pt,
    epochs=EPOCHS_FROZEN, lr=LR_FROZEN, model_name="ResNet18 (Frozen Backbone)"
)

---

# Part 5: Transfer Learning - Full Fine-tuning

**Strategy 2: Full Fine-tuning**
- Start with pre-trained weights
- Train ALL layers (backbone + classifier)
- Use a **lower learning rate** to avoid destroying pre-trained features

```
┌────────────────────────────────┐    ┌─────────────────┐
│   Pre-trained ResNet18         │    │  New Classifier │
│   (TRAINED - small LR)         │ →  │   (TRAINED)     │
│   Adapt features to domain     │    │   8 classes     │
└────────────────────────────────┘    └─────────────────┘
```

### Exercise 3: Create a Full Fine-tuning Model

Load a new pre-trained ResNet18 where ALL parameters are trainable.

In [ ]:
# TODO: Load pre-trained ResNet18 for full fine-tuning
model_finetune = models.resnet18(weights='IMAGENET1K_V1')

# TODO: Replace the final layer (but DON'T freeze any parameters)
# <-- Add your code here!



model_finetune = model_finetune.to(device)

# Check parameters
total_params = sum(p.numel() for p in model_finetune.parameters())
trainable_params = sum(p.numel() for p in model_finetune.parameters() if p.requires_grad)

print(f"ResNet18 (Full Fine-tuning)")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Trainable ratio: {100*trainable_params/total_params:.2f}%")

In [ ]:
# Train with full fine-tuning
# Note: Use LOWER learning rate to preserve pre-trained features
EPOCHS_FINETUNE = 15
LR_FINETUNE = 0.0001  # 10x smaller than frozen backbone!

history_finetune = train_model(
    model_finetune, train_loader_pt, val_loader_pt,
    epochs=EPOCHS_FINETUNE, lr=LR_FINETUNE, model_name="ResNet18 (Full Fine-tuning)"
)

### Question 5: Comparing Training Strategies

1. Why do we use a **lower learning rate** for full fine-tuning compared to frozen backbone?

2. Full fine-tuning takes longer per epoch. Why?

3. When would you choose frozen backbone vs full fine-tuning?

---

# Part 6: Model Comparison

In [ ]:
# Compare all models
histories = [
    (history_scratch, 'Simple CNN (scratch)'),
    (history_frozen, 'ResNet18 (frozen)'),
    (history_finetune, 'ResNet18 (fine-tuned)'),
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

colors = ['#1f77b4', '#ff7f0e', '#2ca02c']

for (history, label), color in zip(histories, colors):
    ax1.plot(history['val_acc'], label=label, linewidth=2, color=color)
    ax2.plot(history['val_loss'], label=label, linewidth=2, color=color)

ax1.set_xlabel('Epoch', fontsize=11)
ax1.set_ylabel('Validation Accuracy (%)', fontsize=11)
ax1.set_title('Model Comparison - Validation Accuracy', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel('Epoch', fontsize=11)
ax2.set_ylabel('Validation Loss', fontsize=11)
ax2.set_title('Model Comparison - Validation Loss', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Final comparison table
print("\n" + "=" * 70)
print("                      FINAL MODEL COMPARISON")
print("=" * 70)
print(f"{'Model':<30} {'Best Val Acc':<15} {'Final Val Acc':<15}")
print("-" * 70)

for history, label in histories:
    best_acc = max(history['val_acc'])
    final_acc = history['val_acc'][-1]
    print(f"{label:<30} {best_acc:<15.2f} {final_acc:<15.2f}")

print("-" * 70)

## Evaluate Best Model on Test Set

In [ ]:
# Evaluate the fine-tuned model on test set
criterion = nn.CrossEntropyLoss()
test_loss, test_acc = evaluate(model_finetune, test_loader_pt, criterion, device)

print(f"\nTest Set Performance (ResNet18 Fine-tuned):")
print(f"  Test Loss: {test_loss:.4f}")
print(f"  Test Accuracy: {test_acc:.2f}%")

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get predictions
model_finetune.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader_pt:
        inputs = inputs.to(device)
        outputs = model_finetune(inputs)
        _, preds = outputs.max(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.squeeze().numpy())

# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)

# Plot
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names, ax=ax)
ax.set_xlabel('Predicted', fontsize=11)
ax.set_ylabel('Actual', fontsize=11)
ax.set_title('Confusion Matrix - ResNet18 Fine-tuned', fontsize=12, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Classification report
print("\nClassification Report:")
print(classification_report(all_labels, all_preds, target_names=class_names))

### Question 6: Final Analysis

1. Which cell types are most often **confused** with each other? Does this make biological sense?

2. Looking at the precision and recall per class, which classes are **hardest to classify**?

3. Is the model performance **good enough for clinical use**? What accuracy would you need?

4. What further improvements could you make? (More data? Better augmentation? Larger model?)

---

# Summary

## What We Built

```
Approach 1: Simple CNN from Scratch
    28x28 → Custom CNN (374K params) → 8 classes
    
Approach 2: Frozen Backbone
    224x224 → ResNet18 (frozen) → Linear(512, 8) → 8 classes
    
Approach 3: Full Fine-tuning  
    224x224 → ResNet18 (all trainable) → Linear(512, 8) → 8 classes
```

## Key Takeaways

| Approach | Pros | Cons | When to Use |
|----------|------|------|-------------|
| **From Scratch** | Full control, small model | Needs lots of data | Large datasets, unique domains |
| **Frozen Backbone** | Fast, little data needed | Limited adaptation | Quick prototyping, small datasets |
| **Full Fine-tuning** | Best performance | Slower, may overfit | When you need max accuracy |

## Transfer Learning Works Because:

1. **Low-level features transfer**: Edges, textures, shapes are universal
2. **Pre-training on large data**: ImageNet provides rich representations
3. **Domain adaptation**: Fine-tuning adapts features to your specific task

## This Pattern Works Across Domains

| Domain | Pre-trained Model | Application |
|--------|-------------------|-------------|
| Medical imaging | ResNet, EfficientNet | X-ray, CT, microscopy |
| Satellite imagery | ResNet | Land use, crop analysis |
| Scientific images | ResNet, ViT | Cell segmentation, particle physics |

---

## Reflection Questions

1. **In your research**, what imaging tasks could benefit from transfer learning?

2. **What pre-trained models** might be most relevant to your domain? (Medical? Satellite? General?)

3. **How much labeled data** do you have? Would frozen backbone or fine-tuning be more appropriate?

4. **What domain-specific augmentations** would make sense for your data?

5. **What accuracy level** would be needed for your application to be useful?

---

## Bonus Exercises

If you have time, try these extensions:

1. **Try a different architecture**: Replace ResNet18 with EfficientNet-B0

2. **Learning rate scheduling**: Implement differential learning rates (lower for backbone, higher for classifier)

3. **More augmentation**: Add CutOut, MixUp, or other modern augmentation techniques

4. **Class imbalance**: Handle the imbalanced classes with weighted loss or oversampling

In [ ]:
# Bonus: Try EfficientNet-B0
import timm

# TODO: Load EfficientNet-B0 and fine-tune it
# model_efficient = timm.create_model('efficientnet_b0', pretrained=True, num_classes=8)
# model_efficient = model_efficient.to(device)

# Your code here...